In [1]:
from chaining import get_chain
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    context_recall,
    answer_relevancy,
    context_precision,
)
from chaining import get_chain, get_retriever
from verification_datas import datas

/Users/tera/Desktop/workspace/skn_21/project/project_03_1team_repo/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/kz/rts6j3mj2nj3xm6c6kf4t1h80000gn/T/ipykernel_35438/458544797.py:4: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import (
/var/folders/kz/rts6j3mj2nj3xm6c6kf4t1h80000gn/T/ipykernel_35438/458544797.py:4: DeprecationWarning: Importing context_recall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import context_recall
  from ragas.metrics import (
/var/folders/kz/rts6j3mj2nj3xm6c6kf4t

In [2]:
chain = get_chain()
retriever = get_retriever()

questions = [x["question"] for x in datas]
targets = [x["answer"] for x in datas]

dataset={
    "reference": targets,
    "user_input": questions,
    "retrieved_contexts": [],
    "response": []
}

/Users/tera/Desktop/workspace/skn_21/project/project_03_1team_repo/.venv/lib/python3.13/site-packages/langchain_qdrant/qdrant.py:465: UserWarning: Api key is used with an insecure connection.
  client = QdrantClient(


In [5]:
import time
from tqdm import tqdm

for i, query in tqdm(enumerate(questions), 'verification'):
    response = chain.invoke(query)
    docs = retriever.invoke(query)
    contexts = [doc.page_content for doc in docs]

    dataset["response"].append(response)
    dataset["retrieved_contexts"].append(contexts)

result = evaluate(
    dataset=Dataset.from_dict(dataset),
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall, # 실답안이 있는 경우에만
    ],
)

print(result)

verification: 5it [14:55, 179.11s/it]
Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Exception raised in Job[1]: AttributeError('OpenAIEmbeddings' object has no attribute 'embed_query')
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Exception raised in Job[5]: AttributeError('OpenAIEmbeddings' object has no attribute 'embed_query')
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Exception raised in Job[9]: AttributeError('OpenAIEmbeddings' object has no attribute 'embed_query')
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Exception raised in Job[13]: AttributeError('OpenAIEmbeddings' object has no attribute 'embed_query')
Exception raised in Job[16]: InstructorRetryException(<failed_attempts>

<generation number="1">
<exception>
    The output is incomplete due to a max_tokens length limit.
</excep

{'faithfulness': 0.6954, 'answer_relevancy': nan, 'context_precision': 0.6317, 'context_recall': 0.7667}


In [6]:
from ragas.metrics import (
    LLMContextRecall, # Context Recall 평가함수
    LLMContextPrecisionWithReference, # Context Precision
    Faithfulness, 
    AnswerRelevancy
)
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

from langchain_openai import ChatOpenAI, OpenAIEmbeddings

eval_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1"))
eval_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-large"))

metrics = [
    LLMContextRecall(llm=eval_llm),
    LLMContextPrecisionWithReference(llm=eval_llm),
    Faithfulness(llm=eval_llm),
    AnswerRelevancy(llm=eval_llm, embeddings=eval_embeddings)
]

# 평가를 진행 - evaluate() 함수 이용
eval_result = evaluate(dataset=Dataset.from_dict(dataset), metrics=metrics)

/var/folders/kz/rts6j3mj2nj3xm6c6kf4t1h80000gn/T/ipykernel_35438/2928539308.py:1: DeprecationWarning: Importing LLMContextRecall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import LLMContextRecall
  from ragas.metrics import (
/var/folders/kz/rts6j3mj2nj3xm6c6kf4t1h80000gn/T/ipykernel_35438/2928539308.py:1: DeprecationWarning: Importing LLMContextPrecisionWithReference from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import LLMContextPrecisionWithReference
  from ragas.metrics import (
/var/folders/kz/rts6j3mj2nj3xm6c6kf4t1h80000gn/T/ipykernel_35438/2928539308.py:1: DeprecationWarning: Importing Faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import Faithfulness
 

In [7]:
eval_result

{'context_recall': 0.1333, 'llm_context_precision_with_reference': 0.0667, 'faithfulness': 0.6825, 'answer_relevancy': 0.1197}